In [70]:
import math
usuarios = [
    {
        "id":"13", 
        "media":3.5,
        "user_similarity": [],
        "filmes":[
                {"id":20, "avaliacao":4, "ano":2012, "mes": 12, "dia": 3}]
     },
    {
        "id":"14", 
        "media":3.5,
        "user_similarity": [],
        "filmes":[
                {"id":20, "avaliacao":4, "ano":2012, "mes": 12, "dia": 3}]
     },
    {
        "id":"16", 
        "media":3.5,
        "user_similarity": [],
        "filmes":[
                {"id":20, "avaliacao":4, "ano":2012, "mes": 12, "dia": 3}]
     },
    {
        "id":"17", 
        "media":1.5,
        "user_similarity": [],
        "filmes":[
                {"id":40, "avaliacao":1, "ano":2012, "mes": 12, "dia": 3}]
     }
    
    
]

print(usuarios, "\n\n")


[{'id': '13', 'media': 3.5, 'user_similarity': [], 'filmes': [{'id': 20, 'avaliacao': 4, 'ano': 2012, 'mes': 12, 'dia': 3}]}, {'id': '14', 'media': 3.5, 'user_similarity': [], 'filmes': [{'id': 20, 'avaliacao': 4, 'ano': 2012, 'mes': 12, 'dia': 3}]}, {'id': '16', 'media': 3.5, 'user_similarity': [], 'filmes': [{'id': 20, 'avaliacao': 4, 'ano': 2012, 'mes': 12, 'dia': 3}]}, {'id': '17', 'media': 1.5, 'user_similarity': [], 'filmes': [{'id': 40, 'avaliacao': 1, 'ano': 2012, 'mes': 12, 'dia': 3}]}] 




Calculo da similadiridade

In [72]:
for i in range(len(usuarios)):
    for j in range(i + 1,len(usuarios)):
        filmes_i = usuarios[i]["filmes"]
        filmes_j = usuarios[j]["filmes"]
        somatorio_1 = 0
        somatorio_2 = 0
        somatorio_3 = 0
        m = 0
        for f1 in filmes_i:
            f2 = list(filter(lambda y: y["id"] == f1["id"],filmes_j))
            if f2:
                f2 = f2[0]
                s1 = (f1["avaliacao"] - usuarios[i]["media"])
                s2 = (f2["avaliacao"] - usuarios[j]["media"])
                somatorio_1 += s1*s2
                somatorio_2 += math.pow(s1, 2)
                somatorio_3 += math.pow(s2, 2)
                m = m + 1
        raiz = math.sqrt( somatorio_2 * somatorio_3)
        if (raiz): #verifica divizao por zero
            similaridade = somatorio_1 / raiz
        else:
            similaridade = 0
        if(m < 50):
            similaridade = similaridade / (50-m)        
        if(similaridade > 0):
            usuarios[i]["user_similarity"].append((usuarios[j]["id"], similaridade))
            usuarios[j]["user_similarity"].append((usuarios[i]["id"], similaridade))           
print(usuarios[0]["user_similarity"])
            
        

[('14', 0.02040816326530612), ('16', 0.02040816326530612), ('14', 0.02040816326530612), ('16', 0.02040816326530612)]
